In [1]:
%run "model_random.ipynb"

In [2]:
# 读取生成的数据
ds_block = xr.open_dataset(
   "Data/block.nc")
ds_full = xr.open_dataset(
   "Data/full.nc")
# 将生成的数据转化为用于神经网络训练的数据集格式
BLOCK_SIZE, BATCH_SIZE = 2, 4
dens_full, disp_slice, outputs = extract_data(ds_full, ds_block)
dataset = tf.data.Dataset.from_tensor_slices(({"dens_input": dens_full, "disp_input": disp_slice}, outputs))
train_dataset, _, test_dataset = get_dataset(dataset, dens_full.shape[0], batch_size=BATCH_SIZE)

# 初始化神经网络模型
num_of_layers_cnn, num_of_layers_fnn =10, 10
cnn = cnn_model(num_of_layers=num_of_layers_cnn)
fnn = fnn_model(block_size=BLOCK_SIZE, num_of_layers=num_of_layers_fnn)

In [3]:
# 训练神经网络
test_losses, val_losses = custom_train(block_size=BLOCK_SIZE,cnn_model=cnn,fnn_model=fnn,\
                        loss_function=tf.keras.losses.MeanAbsoluteError(),\
                        num_epoch=20,optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),\
                        batched_train=train_dataset, batched_test=test_dataset, num_of_layers_1=num_of_layers_cnn, num_of_layers_2=num_of_layers_fnn,save_model=False, 
                        casename='test')